In [1]:
from typing import *
import ctypes as C
import struct
import time
import numpy as np
import numpy.random as random

SEGMENTS = {
    'us': [
      { 'name': '.data', 'virtual_address': 1302528, 'virtual_size': 2836576 },
      { 'name': '.bss', 'virtual_address': 14045184, 'virtual_size': 4897408 },
    ],
    'jp': [
      { 'name': '.data', 'virtual_address': 1294336, 'virtual_size': 2406112 },
      { 'name': '.bss', 'virtual_address': 13594624, 'virtual_size': 4897632 },
    ],
}

class Game:
    def __init__(self, version, dll_path):
        self.version = version
        self.dll = C.cdll.LoadLibrary(dll_path)
        self.dll.sm64_init()
        self.segments = SEGMENTS[version]

    def advance_frame(self):
        self.dll.sm64_update()

    def alloc_slot(self):
        buffers = []
        for segment in self.segments:
            buffers.append(C.create_string_buffer(segment['virtual_size']))
        return buffers

    def save_state(self, slot):
        for segment, buffer in zip(self.segments, slot):
            C.memmove(buffer, self.dll._handle + segment['virtual_address'], segment['virtual_size'])

    def load_state(self, slot):
        for segment, buffer in zip(self.segments, slot):
            C.memmove(self.dll._handle + segment['virtual_address'], buffer, segment['virtual_size'])

    def addr(self, symbol):
        return C.addressof(C.c_uint32.in_dll(self.dll, symbol))

def ptr(addr, type):
    return C.cast(addr, C.POINTER(type))

def load_m64(filename):
    frames = []
    with open(filename, 'rb') as f:
        f.seek(0x400)
        while True:
            try:
                buttons = struct.unpack('>H', f.read(2))[0]
                stick_x = struct.unpack('=b', f.read(1))[0]
                stick_y = struct.unpack('=b', f.read(1))[0]
            except struct.error:
                break
            frames.append((buttons, stick_x, stick_y))
    return frames

def set_inputs(game, inputs):
    buttons, stick_x, stick_y = inputs
    ptr(game.addr('gControllerPads') + 0, C.c_uint16)[0] = buttons
    ptr(game.addr('gControllerPads') + 2, C.c_int8)[0] = stick_x
    ptr(game.addr('gControllerPads') + 3, C.c_int8)[0] = stick_y



In [2]:
#hack way to do this. I'm sure there is something better but
#I don't know what I'm doing
def copy_object(source_obj_num, to_obj_num):
    #If I start copying at 30 or less, I get access violation errors.
    for i in range(40, 1392//4):
        to_ptr = ptr(game.addr('gObjectPool') + to_obj_num*1392 + i*4, C.c_float)
        source_ptr = ptr(game.addr('gObjectPool') + source_obj_num*1392 + i*4, C.c_float)
        to_ptr[0] = source_ptr[0]

In [3]:
################################################################################
game = Game('jp', '../libsm64/sm64_jp.dll')
m64 = load_m64('1Key_4_21_13_Padded.m64')

bully_slot_order = [27] + list(range(27)) + list(range(28, 83))
states_by_numbullies = [game.alloc_slot() for i in range(80)]

for frame in range(len(m64)):
    set_inputs(game, m64[frame])
    game.advance_frame()

    num_stars = ptr(game.addr('gMarioStates') + 230, C.c_int16)[0]
    
    if (frame % 1000 == 0): 
        print("Frame %05d stars %02d" % (frame, num_stars))
        
    if (frame == 3286):
        
        for obj in range(108):
            #don't deactivate the bully, mario, or the tilting pyramid platforms
            if obj in [27, 89, 83, 84]:
                continue
            #seems to be either 48 or 180
            active_flag = ptr(game.addr('gObjectPool') + obj*1392 + 180, C.c_short)
            active_flag[0] = active_flag[0] & 0xFFFE
        for extra_bullies in range(0, 80):
            game.save_state(states_by_numbullies[extra_bullies])
            source_num = 27
            to_num = bully_slot_order[extra_bullies + 1]
            copy_object(source_num, to_num)
        break

Frame 00000 stars 00
Frame 01000 stars 00
Frame 02000 stars 00
Frame 03000 stars 00


In [4]:
#game.load_state(states_by_numbullies[1])
#print('here1')
#game.advance_frame()
#print('here2')
#copy_object(27, 26)
#print('here3')
#game.advance_frame()
#print('here4')

In [5]:
# Pointers and definitions.
mario_x = ptr(game.addr('gMarioStates') + 60, C.c_float)
mario_y = ptr(game.addr('gMarioStates') + 64, C.c_float)
mario_z = ptr(game.addr('gMarioStates') + 68, C.c_float)

start_bully_pos = (-2236, -2950, -566)
initial_yaw = 355


In [6]:
def f2i(x):
  return struct.unpack('>l', struct.pack('>f', x))[0]

def i2f(x):
  return struct.unpack('>f', struct.pack('>i', x))[0]

In [7]:
#I think there should be 19 solns
bully_x_ptrs, bully_y_ptrs, bully_z_ptrs, bully_hspd_ptrs, bully_yaw_1_ptrs, bully_yaw_2_ptrs = [dict() for i in range(6)]

for bully in range(80):
    bully_x_ptrs[bully] = ptr(game.addr('gObjectPool') + bully_slot_order[bully]*1392 + 240, C.c_float)
    bully_y_ptrs[bully] = ptr(game.addr('gObjectPool') + bully_slot_order[bully]*1392 + 244, C.c_float)
    bully_z_ptrs[bully] = ptr(game.addr('gObjectPool') + bully_slot_order[bully]*1392 + 248, C.c_float)
    bully_hspd_ptrs[bully] = ptr(game.addr('gObjectPool') + bully_slot_order[bully]*1392 + 264, C.c_float)
    bully_yaw_1_ptrs[bully] = ptr(game.addr('gObjectPool') + bully_slot_order[bully]*1392 + 280, C.c_uint16)
    bully_yaw_2_ptrs[bully] = ptr(game.addr('gObjectPool') + bully_slot_order[bully]*1392 + 292, C.c_uint16)
 
#There's supposed to be a solution, but it doesn't find one,
#even the first time through where there shouldn't have been any hacks.
test_speed = 3209466.0
for num_bullies in range(1, 80):
    total_solns = 0
    start_time = time.time()
    backup = states_by_numbullies[num_bullies - 1]
    
    batch = 0
    while batch*num_bullies < 65536:
        game.load_state(backup)
        for bully_num in range(num_bullies):
            bully_x_ptrs[bully_num][0] = start_bully_pos[0]
            bully_y_ptrs[bully_num][0] = start_bully_pos[1]
            bully_z_ptrs[bully_num][0] = start_bully_pos[2]
            bully_hspd_ptrs[bully_num][0] = test_speed
            bully_yaw_1_ptrs[bully_num][0] = batch*num_bullies + bully_num
            bully_yaw_2_ptrs[bully_num][0] = batch*num_bullies + bully_num
            #print(batch*num_bullies + bully_num)
        for iter_frame in range(21):
            mario_x[0] = -1945
            mario_y[0] = -2918
            mario_z[0] = -715
            game.advance_frame()
            for bully_num in range(num_bullies):
                new_bully_pos = (bully_x_ptrs[bully_num][0], bully_y_ptrs[bully_num][0], bully_z_ptrs[bully_num][0])

                dist = ((new_bully_pos[0] - start_bully_pos[0])**2 +
                        #(new_bully_pos[1] - start_bully_pos[1])**2 +
                        (new_bully_pos[2] - start_bully_pos[2])**2)**.5
            
                if dist > 200 and dist < 1000:
                    #print(batch*num_bullies + bully_num)
                    total_solns += 1
        batch += 1
    end_time = time.time()
    print('Seconds elapsed for {0} bullies: {1}, found {2} solns'.format(num_bullies, time.time() - start_time, total_solns))

Seconds elapsed for 1 bullies: 58.86417031288147, found 151 solns
Seconds elapsed for 2 bullies: 30.308781147003174, found 151 solns
Seconds elapsed for 3 bullies: 20.710633993148804, found 151 solns
Seconds elapsed for 4 bullies: 16.003581047058105, found 151 solns
Seconds elapsed for 5 bullies: 13.204954624176025, found 151 solns
Seconds elapsed for 6 bullies: 11.449562549591064, found 151 solns
Seconds elapsed for 7 bullies: 9.940223693847656, found 151 solns
Seconds elapsed for 8 bullies: 8.900991916656494, found 151 solns
Seconds elapsed for 9 bullies: 8.107814073562622, found 151 solns
Seconds elapsed for 10 bullies: 7.427661895751953, found 151 solns
Seconds elapsed for 11 bullies: 6.8635358810424805, found 151 solns
Seconds elapsed for 12 bullies: 6.378427505493164, found 151 solns
Seconds elapsed for 13 bullies: 6.059355735778809, found 151 solns
Seconds elapsed for 14 bullies: 5.763289451599121, found 151 solns
Seconds elapsed for 15 bullies: 5.48722767829895, found 151 solns

In [8]:
2176/19

114.52631578947368